In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('./csv/scrape_run1_data_scientist.csv')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,-1,Position Summary...\nWhat you'll do...\nData S...,3.3,Walmart\n3.3,NaN,"Sunnyvale, CA",10000+ Employees,1962,Company - Public,General Merchandise & Superstores,Retail & Wholesale,$10+ billion (USD)
1,Data Scientist,-1,Position Summary...\nWhat you'll do...\nData S...,3.3,Walmart\n3.3,NaN,"Sunnyvale, CA",10000+ Employees,1962,Company - Public,General Merchandise & Superstores,Retail & Wholesale,$10+ billion (USD)
2,Data Scientist,-1,Come join a growing financial technology compa...,4.4,Continental Finance Company\n4.4,NaN,"Wilmington, DE",51 to 200 Employees,2005,Company - Private,Banking & Lending,Financial Services,$5 to $25 million (USD)
3,Data Analyst,-1,Job Description: Data Analyst\n\nWhy Guarantee...,2.9,Guaranteed Rate\n2.9,NaN,Remote,5001 to 10000 Employees,2000,Company - Private,Banking & Lending,Financial Services,$500 million to $1 billion (USD)
4,"Data Scientist, Marketing & Online (Remote)",125000,Position Purpose:\nThe Data Scientist is respo...,3.8,The Home Depot\n3.8,NaN,"Atlanta, GA",10000+ Employees,1978,Company - Public,Home Furniture & Housewares Stores,Retail & Wholesale,$10+ billion (USD)


In [4]:
df.dtypes

Job Title             object
Salary Estimate        int64
Job Description       object
Rating               float64
Company Name          object
Division              object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [5]:
# Identify if salary is hourly/annual rate, convert to annual
# Change salary and rating type to float 

In [6]:
# import re
# re.sub('[^0-9]','', a)
# df['Rating'] = df['Rating'].astype(str).astype(float)
# cols = ['Job Title', 'Job Description', 'Company Name', 'Division', 'Location', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue']
# df[cols] = df[cols].astype(str)